In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%matplotlib notebook
%use gromacs-2018.4
%use pymol-1.8.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)



HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''

frame_width = 12
spec_height = 12
btn_width = 2.75

def format_width(w):
    return str(w)+'cm'

In [ ]:
class Spectrum:
    def __init__(self, line, xdata, ydata, fname, prefix, checkbox, colorpicker):
        self.line = line
        self.xdata = xdata.copy()
        self.ydata = ydata.copy()
        self.fname = fname
        self.prefix = prefix
        self.cb = checkbox
        self.cp = colorpicker

def on_color_pick(change):
    global SpecList
    cp = change['owner']
    for spec in SpecList:
        if spec.cp==cp:
            spec.line.set_color(cp.value)

def newcp(line):
    picker = widgets.ColorPicker(
        concise=True,
        description='',
        value=line.get_color(),
        disabled=False
        )
    picker.observe(on_color_pick)
    return picker

def on_check_change(change):
    cb = change['owner']
    for spec in SpecList:
        if spec.cb==cb:
            spec.line.set_visible(cb.value)
            update_spec_lims()
    return 0

def newcb(prefix):
    check = widgets.Checkbox(
        description=prefix,
        value=True,
    )
    check.observe(on_check_change)
    return check

load_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=False,
    layout = Layout(width=format_width(2*btn_width))
)

refresh_button = widgets.Button(
    description='Refresh List',
    layout=Layout(width=format_width(btn_width))
)

load_button = widgets.Button(
    description='Load',
    layout=Layout(width=format_width(btn_width))
)


def refresh_onclick(b):
    out = !{'ls spec/*_abs.txt'}
    prefList = []
    for line in out:
        if line[-4:]=='.txt':
            prefList.append(line.split('/')[-1][0:-4])
    load_drop.options = prefList
    if len(prefList)>0:
        load_drop.value = prefList[0]

refresh_button.on_click(refresh_onclick)

def load_button_onclick(b):
    global SpecList
    global SpecBoxes
    global SpecSelect
    global SpecSelect2
    
    # If no parameter files are available, abort
    if len(load_drop.options)==0:
        return 0
    
    prefix = load_drop.value
    fullname = 'spec/'+prefix+'.txt'
    data = np.loadtxt(fullname)
    if np.shape(data)[1]!=2:
        print('Error reading from file ' + fullname)
    else:
        PrefList = list()
        for spec in SpecList:
            PrefList.append(spec.prefix)
        while PrefList.count(prefix)>0:
            prefix += '_x'
            
        newline, = specax.plot(data[:,0], data[:,1])
        newpicker = newcp(newline)
        newcheck = newcb(prefix)
        SpecList.append(Spectrum(newline, data[:,0], data[:,1], fullname, prefix, newcheck, newpicker))
        SpecBoxes.children = list(SpecBoxes.children) + [widgets.HBox([newcheck, newpicker])]
        update_spec_lims()
        SpecSelect.options = list(SpecSelect.options) + [prefix]
        SpecSelect2.options = list(SpecSelect2.options) + [prefix]
    
load_button.on_click(load_button_onclick)


def build_axes():
#     specax.set_yticks([])
    specax.set_xlabel('$\\bar\\nu$ (cm$^{-1}$)')
    specax2.xaxis.tick_top()
    specax2.set_xlabel('$\\lambda$ (nm)')
#     specax2.format_coord = format_coord
    update_ax2()
    plt.tight_layout()
    plt.show()

def update_ax2():
    global specax2
    specax2.set_xlim(specax.get_xlim())
    xticks2 = (1.0e7)/(specax.get_xticks())
    dlambda = -np.mean(np.diff(xticks2))
    ndec = max(-int(np.ceil(np.log10(dlambda))), 0)
    fmt = "{:." + str(ndec) + "f}"
    xticklabels2 = []
    for tick in xticks2:
        xticklabels2.append(fmt.format(tick))
    
    specax2.set_xticks((1.0e7)/xticks2)
    specax2.set_xticklabels(xticklabels2)
    specax2.set_xlim(specax.get_xlim())
    plt.tight_layout()
    
    
def update_spec_lims():
    
    lines = []
    for spec in SpecList:
        lines.append(spec.line)
    myinf = 1e+100
    mny = myinf
    mxy = -myinf
    for line in lines:
        if line.get_visible()==True:
            mny = min(mny, np.min(line.get_ydata()))
            mxy = max(mxy, np.max(line.get_ydata()))
    if (mny<myinf) and (mxy>-myinf) and (mny!=mxy):
        specax.set_ylim(mny, mxy)
        
    mnx = len(vaxis) - 1
    mxx = 0
    cutoff = 1e-6
    for line in lines:
        if line.get_visible()==True:
            ndcs = np.where(np.abs(line.get_ydata())>cutoff)[0]
            if len(ndcs>0):
                mnx = min(mnx, np.min(ndcs-1))
                mxx = max(mxx, np.max(ndcs+1))
    if mxx>mnx:
        specax.set_xlim(vaxis[mnx], vaxis[mxx])
    update_ax2()
    plt.tight_layout()





norm1_button = widgets.Button(
    description='Peak Norm',
    layout=Layout(width=format_width(btn_width))
)
def spec_norm1(b):
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec.line.set_ydata(spec.line.get_ydata()/np.max(spec.line.get_ydata()))
            update_spec_lims()
            break
norm1_button.on_click(spec_norm1)



normL1_button = widgets.Button(
    description='Sum Norm',
    layout=Layout(width=format_width(btn_width))
)
def spec_normL1(b):
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec.line.set_ydata(spec.line.get_ydata()/np.sum(spec.line.get_ydata()))
            update_spec_lims()
            break
normL1_button.on_click(spec_normL1)


normL2_button = widgets.Button(
    description='RMS Norm',
    layout=Layout(width=format_width(btn_width))
)            
def spec_normL2(b):
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec.line.set_ydata(spec.line.get_ydata()/np.linalg.norm(spec.line.get_ydata()))
            update_spec_lims()
            break
normL2_button.on_click(spec_normL2)            


revert_button = widgets.Button(
    description='Undo All',
    layout=Layout(width=format_width(btn_width))
)            
def spec_revert(b):
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec.line.set_ydata(spec.ydata.copy())
            update_spec_lims()
            break
revert_button.on_click(spec_revert)

delete_button = widgets.Button(
    description='Delete',
    layout=Layout(width=format_width(btn_width))
)            
def spec_delete(b):
    global SpecList
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            NewOpts = list(SpecSelect.options)
            del NewOpts[NewOpts.index(spec.prefix)]
            SpecSelect.options = NewOpts
            SpecSelect2.options = NewOpts
            specax.lines.remove(spec.line)
            BoxList = list(SpecBoxes.children)
            for box in BoxList:
                if box.children[1]==spec.cp:
                    del BoxList[BoxList.index(box)]
                    break
            SpecBoxes.children = BoxList
            del SpecList[SpecList.index(spec)]
            plt.draw()
            update_spec_lims()
            break
delete_button.on_click(spec_delete)

copy_button = widgets.Button(
    description='Copy',
    layout=Layout(width=format_width(btn_width))
)            
def spec_copy(b):
    global SpecList
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            newline, = specax.plot(spec.xdata, spec.ydata)
            newpicker = newcp(newline)
            newcheck = newcb(prefix)
            newspec = Spectrum(newline, spec.xdata, spec.ydata, spec.fname, spec.prefix + "_copy", newcheck, newpicker)
            SpecList.append(newspec)
#             SpecBoxes.append(newpicker)
            SpecBoxes.children = list(SpecBoxes.children) + [widgets.HBox([newcheck, newpicker])]
            NewOpts = list(SpecSelect.options) + [newspec.prefix]
            SpecSelect.options = NewOpts
            SpecSelect2.options = NewOpts
#             plt.draw()
            update_spec_lims()
            break
copy_button.on_click(spec_copy)





save_button = widgets.Button(
    description='Export To:',
    layout=Layout(width=format_width(btn_width))
)
save_name = widgets.Text(
    description='',
    layout=Layout(width=format_width(2*btn_width))
)

def spec_save(b):
    global SpecList
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            fname = 'spec/' + save_name.value
            if fname[-4:]!='.txt':
                fname += '.txt'
            np.savetxt(fname, np.vstack((spec.xdata, spec.line.get_ydata())).T)
            break
save_button.on_click(spec_save)

spec_down = widgets.HTML(HTMLButtonPrompt.format(link='spec/', text='Download'))



add_button = widgets.Button(
    description='Add',
    layout=Layout(width=format_width(btn_width))
)            
def spec_add(b):
    
    # Identify first spectrum
    spec1 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec1 = spec
            break
            
    # Identify second spectrum
    spec2 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect2.value:
            spec2 = spec
            break
    
    # Operate
    if (spec1!=[]) and (spec2!=[]):
        spec1.line.set_ydata(spec1.line.get_ydata() + spec2.line.get_ydata())
        update_spec_lims()
add_button.on_click(spec_add) 



subtract_button = widgets.Button(
    description='Subtract',
    layout=Layout(width=format_width(btn_width))
)            
def spec_subtract(b):
    
    # Identify first spectrum
    spec1 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec1 = spec
            break
            
    # Identify second spectrum
    spec2 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect2.value:
            spec2 = spec
            break
    
    # Operate
    if (spec1!=[]) and (spec2!=[]):
        spec1.line.set_ydata(spec1.line.get_ydata() - spec2.line.get_ydata())
        update_spec_lims()
subtract_button.on_click(spec_subtract) 


multiply_button = widgets.Button(
    description='Multiply',
    layout=Layout(width=format_width(btn_width))
)            
def spec_multiply(b):
    
    # Identify first spectrum
    spec1 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec1 = spec
            break
            
    # Identify second spectrum
    spec2 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect2.value:
            spec2 = spec
            break
    
    # Operate
    if (spec1!=[]) and (spec2!=[]):
        spec1.line.set_ydata(spec1.line.get_ydata()*spec2.line.get_ydata())
        update_spec_lims()
multiply_button.on_click(spec_multiply)  




divide_button = widgets.Button(
    description='Divide',
    layout=Layout(width=format_width(btn_width))
)            
def spec_divide(b):
    
    # Identify first spectrum
    spec1 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect.value:
            spec1 = spec
            break
            
    # Identify second spectrum
    spec2 = []
    for spec in SpecList:
        if spec.prefix==SpecSelect2.value:
            spec2 = spec
            break
    
    # Operate
    if (spec1!=[]) and (spec2!=[]):
        good_ndcs = np.where(np.abs(spec2.line.get_ydata())>0)[0]
        bad_ndcs = np.where(np.abs(spec2.line.get_ydata())==0)[0]
        newdata = spec1.line.get_ydata()
        newdata[good_ndcs] = newdata[good_ndcs]/spec2.line.get_ydata()[good_ndcs]
        newdata[bad_ndcs] = 0.0
        spec1.line.set_ydata(newdata)
        update_spec_lims()
        
divide_button.on_click(spec_divide)



    
SpecSelect = widgets.Dropdown(
    options = [], 
    description = 'Spec1',
    layout=Layout(width=format_width(0.45*frame_width))
)
SpecSelect.style.description_width='1cm'

SpecSelect2 = widgets.Dropdown(
    options = [], 
    description = 'Spec2',
    layout=Layout(width=format_width(0.45*frame_width))
)
SpecSelect2.style.description_width='1cm'

FunBoxLayout = widgets.Layout(
    width='6cm',
    flex_flow = 'row wrap',
#     align_items = 'center',
    
 )
FunBox1 = widgets.Box([norm1_button, normL1_button, normL2_button, revert_button, delete_button, copy_button], layout=FunBoxLayout)
FunBox2 = widgets.Box([add_button, subtract_button, multiply_button, divide_button], layout=FunBoxLayout)

def init_specout():
    global specfig
    global specax
    global specax2
    global SpecList
    with specout:
        %matplotlib notebook
        specfig = plt.figure(figsize=(4.45,3.5))
        specax = plt.gca()
        specax2 = specax.twiny()
        base_line, = specax.plot(vaxis, 0.0*vaxis)
        newpicker = newcp(base_line)
        newcheck = newcb('baseline')
        SpecList.append(Spectrum(base_line, base_line.get_xdata(), base_line.get_ydata(), '', 'baseline', newcheck, newpicker))
#         SpecBoxes.children = list(SpecBoxes.children) + widgets.HBox([newcheck, newpicker])
        build_axes()

frame_layout = Layout(
    flex_flow = 'row',
    width = format_width(frame_width),
    justify_content = 'space-around'
)
SpecList = []
SpecBoxes = widgets.VBox([])
refresh_onclick(0)
specout = widgets.Output(layout=Layout(width=format_width(frame_width),
                                      height=format_width(spec_height)))
left_frame = widgets.VBox([
    widgets.HBox([load_button, load_drop, refresh_button], layout=frame_layout),
    specout,
    widgets.Box([SpecSelect, SpecSelect2], layout=frame_layout),
    widgets.HBox([FunBox1, FunBox2]),
    widgets.Label(layout=Layout(
        width=format_width(2*btn_width),
        height='0.5cm'
    )),
    widgets.HBox([save_button, save_name]), 
    spec_down
])

right_frame = widgets.VBox([SpecBoxes])

display(widgets.HBox([left_frame, right_frame]))
vaxis = np.arange(10000, 18000)
init_specout()